In [1]:
from llama_index.embeddings.ollama import OllamaEmbedding
        
embedding_model =  OllamaEmbedding(model_name="nomic-embed-text:v1.5")

In [2]:
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss
embedding_dimension = 512

fais_index = faiss.IndexFlatL2(embedding_dimension)
    
vector_store = FaissVectorStore(faiss_index=fais_index)#, callback_manager=callback_manager)

In [33]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from transcripts_rag.preprocessing import TextCleaner


pipeline = IngestionPipeline(
        transformations=[
            SentenceSplitter(),
            TextCleaner()
        ],
        vector_store=vector_store,
        )

In [34]:
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata



def generate_tools(base_path, path_list):
        
        docs_list = []
        
        tool_list = []
        
        full_path_list = [base_path+path for path in path_list]
                
        name_list = [x.split('_')[-1].split('.')[0] for x in path_list]
        #name_list = [x.split('_')[0] for x in path_list]
                
        for path in full_path_list:
            doc = SimpleDirectoryReader(
                input_files=[path]
            ).load_data()
                        
            docs_list.append(doc)
            
        for doc, name in zip(docs_list, name_list):
            
            print(name)
            
            nodes = pipeline.run(documents=doc)
                        
            index = VectorStoreIndex(nodes= nodes, embed_model=embedding_model)
                        
            index.storage_context.persist(persist_dir=f"storage/{name}")
            engine = index.as_query_engine(similarity_top_k=3)
                                    
            tool = QueryEngineTool(
                    query_engine=engine,
                    metadata=ToolMetadata(
                        name=f"{name.lower()}_financials",
                        description=(
                            f"Provides information about {name} financials. "
                            "Use a detailed plain text question as input to the tool. "
                            "The input is used to power a semantic search engine."
                        ),
                    ),
                )
                        
            tool_list.append(tool)
            
        return tool_list
            

In [35]:
import os

check_ls = os.listdir('../data/')[-2:]
        
query_engine_tools = generate_tools('../data/', check_ls)

ABNB
ARM


In [37]:
from llama_index.packs.agents_coa import CoAAgentPack
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
pack = CoAAgentPack(tools=query_engine_tools, llm=llm)

In [39]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x16bca5ed0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x16bca5540>, model_name='llama-3.1-8b-instant', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [38]:
import nest_asyncio
nest_asyncio.apply()
response = pack.run("How did lyft revenue growth compare to uber?")

AttributeError: 'NoneType' object has no attribute 'as_trace'